In [16]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import KFold
import numpy as np

def load_dataset():
    (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
    
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):

    train_norm = train.astype('float32')
    test_norm = test.astype('float32')

    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0

    return train_norm, test_norm

def define_model(depth):
    model = Sequential()
    # Add the first layer to specify input shape
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(trainX.shape[1], trainX.shape[2], trainX.shape[3])))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))  # Adding dropout after the initial conv block
    
    # Dynamically add more layers based on the depth parameter
    for _ in range(1, depth):
        model.add(Conv2D(32*(2**_), (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))  # Adding dropout after each conv block

    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))  # Adding dropout before the final dense layer
    model.add(Dense(10, activation='softmax'))  # Assuming 10 classes
    
    # Compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model_kfold(X, y, n_folds=5, depth=2):
    scores, histories = list(), list()
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    for train_ix, test_ix in kfold.split(X):
        model = define_model(depth)
        # Select rows for train and test
        trainX, trainY, testX, testY = X[train_ix], y[train_ix], X[test_ix], y[test_ix]
        # Fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        # Evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        scores.append(acc)
        histories.append(history)
    return scores, histories

# plot diagnostic learning curves
def summarize_diagnostics(histories):
     for i in range(len(histories)):
         # plot loss
         pyplot.subplot(211)
         pyplot.title('Cross Entropy Loss')
         pyplot.plot(histories[i].history['loss'], color='blue', label='train')
         pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
         # plot accuracy
         pyplot.subplot(212)
         pyplot.title('Classification Accuracy')
         pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
         pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
         pyplot.show()

# summarize model performance
def summarize_performance(scores):
     # print summary
     print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
     # box and whisker plots of results
     pyplot.boxplot(scores)
     pyplot.show()

# run the test harness for evaluating a model
def run_test_harness():
     # load dataset
     trainX, trainY, testX, testY = load_dataset()
     # prepare pixel data
     trainX, testX = prep_pixels(trainX, testX)
     # evaluate model
     scores, histories = evaluate_model(trainX, trainY)
     # learning curves
     summarize_diagnostics(histories)
     # summarize estimated performance
     summarize_performance(scores)
 
# Example usage
# Load and prepare data
trainX, trainY, testX, testY = load_dataset()  # Implement this
trainX, testX = prep_pixels(trainX, testX)  # Implement this

# Evaluate model with different depths using k-fold cross-validation
depths = [2, 3, 4]  # Example depths to test
for depth in depths:
    print(f'Training for Depth {depth}')
    scores, histories = evaluate_model_kfold(trainX, trainY, n_folds=5, depth=depth)
    print('Depth: %d, Accuracy: %.3f%%' % (depth, np.mean(scores)*100))


Training for Depth 2
> 91.233
> 91.267
> 91.083
> 90.450
> 89.475
Depth: 2, Accuracy: 90.702%
Training for Depth 3
> 90.292
> 90.200
> 90.050
> 90.008
> 88.850
Depth: 3, Accuracy: 89.880%
Training for Depth 4
> 89.650
> 90.150
> 89.767
> 90.000
> 90.158
Depth: 4, Accuracy: 89.945%
